In [1]:
pip install torch transformers sentencepiece pytorch_lightning datasets tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import pytorch_lightning as pl
from transformers import CamembertTokenizer, AdamW, CamembertForSequenceClassification, CamembertConfig, CamembertForTokenClassification
import pandas as pd
from transformers import CamembertForTokenClassification, CamembertConfig, AutoModelForTokenClassification, CamembertPreTrainedModel
from pprint import pprint
import functools
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import pytorch_lightning as pl
from transformers import AutoModelForSequenceClassification, CamembertForMaskedLM, AutoTokenizer, AutoConfig
from datasets import load_dataset
from sklearn.metrics import confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm.notebook import tqdm
import pandas as pd
from datasets import Dataset
import numpy as np
import random

data = pd.read_csv("final-keys-dataset.csv")

# Drop the rows with missing values
data = data.dropna()

# Remove the rows with empty strings

data = data[data['keywords'] != '']

# Remove duplicates

data = data.drop_duplicates(subset=['keywords'])

# Convert the keywords into a string of words separated by spaces instead of a list

data['keywords'] = data['keywords'].apply(lambda x: ' '.join(x.split(',')))

# Remove the brackets

data['keywords'] = data['keywords'].apply(lambda x: x.replace('[', '').replace(']', ''))

# Remove the quotes

data['keywords'] = data['keywords'].apply(lambda x: x.replace("'", ''))

# Get only 6 keywords, if there are more than 6. If there are less than 6, drop the row

data['keywords'] = data['keywords'].apply(lambda x: ' '.join(x.split()[:6]))



from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("yanekyuk/camembert-keyword-extractor")

model = AutoModelForTokenClassification.from_pretrained("yanekyuk/camembert-keyword-extractor")


In [3]:
# Create a list of dictionaries where each dictionary represents a document and its corresponding keywords
datax = []
for index, row in data.iterrows():
    doc = {
        "text": row["text"],
        "keywords": row['keywords'].split()
    }
    datax.append(doc)

In [4]:
print(datax[0])

{'text': "Pour bien maîtriser la logique de l'enregistrement des opérations comptables, on va faire cet exercice. On a un bilan initial ici et on a plusieurs opérations à enregistrer. Je vais ajouter ici le compte de résultats parce qu'on va l'utiliser pendant l'enregistrement comptable des opérations. En fait, j'ai déjà répondu à ce même exercice en utilisant les schémas en T. Dans cet exercice-là, cette fois-ci, on va utiliser simplement l'impact qu'il y aurait sur le bilan initial et le compte de résultats directement sans passer par les schémas en T. On va garder cette image pour vérification après. Donc, on devrait arriver au même bilan final qu'ici. On commence tout de suite par la première opération, achat de marchandise 2300 € hors TVA. Le taux de TVA est de 21 %. Dans cette opération, on a une augmentation de charge au niveau achat de marchandise. Donc, on va augmenter le compte achat de marchandise de 2300 € et on reçoit une facture à la suite de cet achat du fournisseur. Eh 

In [73]:
class TokenClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, label2id):
        self.data = data
        self.tokenizer = tokenizer
        self.label2id = label2id
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        doc = self.data[index]
        text = str(doc["text"])
        text = text[:300]
        tokens = self.tokenizer.tokenize(text, truncation=True, padding=True, max_length=128*2)

        labels = ["O"] * len(tokens)

        for keyword in doc["keywords"]:
            keyword_tokens = self.tokenizer.tokenize(keyword,truncation=True, padding=True, max_length=128*2)
            keyword_start_idx = -1
            for i in range(len(tokens) - len(keyword_tokens) + 1):
                if tokens[i:i+len(keyword_tokens)] == keyword_tokens:
                    keyword_start_idx = i
                    break
            if keyword_start_idx >= 0:
                labels[keyword_start_idx] = "B-KEY"
                for i in range(keyword_start_idx+1, keyword_start_idx+len(keyword_tokens)):
                    labels[i] = "I-KEY"

        label_ids = [self.label2id[label] for label in labels]

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_ids)

        return torch.tensor(input_ids), torch.tensor(attention_mask), torch.tensor(label_ids)


In [74]:
batch_size = 8

def collate_fn(batch):
    input_ids, attention_masks, labels = zip(*batch)
    max_len = max(len(ids) for ids in input_ids)
    input_ids_tensor = torch.zeros((batch_size, max_len), dtype=torch.long)
    attention_masks_tensor = torch.zeros((batch_size, max_len), dtype=torch.long)
    labels_tensor = torch.zeros((batch_size, max_len), dtype=torch.long)
    for i, ids in enumerate(input_ids):
        input_ids_tensor[i, :len(ids)] = torch.tensor(ids)
        attention_masks_tensor[i, :len(ids)] = torch.ones(len(ids), dtype=torch.long)
        labels_tensor[i, :len(ids)] = torch.tensor(labels[i])
    input_ids_tensor = torch.nn.utils.rnn.pad_sequence(input_ids_tensor, batch_first=True, padding_value=-100)
    labels_tensor = torch.nn.utils.rnn.pad_sequence(labels_tensor, batch_first=True, padding_value=-100)
    return input_ids_tensor, attention_masks_tensor, labels_tensor


dataset = TokenClassificationDataset(datax, tokenizer,{"B-KEY": 1, "I-KEY": 2, "O": 0})
# Split the dataset into train and validation sets

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create the data loaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
#dataset = TokenClassificationDataset(datax, tokenizer,{"B-KEY": 1, "I-KEY": 2, "O": 0})
#dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [61]:
for d in val_dataloader:
  print(d[0].shape)
  break

torch.Size([8, 120])


<ipython-input-60-e9846bbc52d0>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_tensor[i, :len(ids)] = torch.tensor(ids)
<ipython-input-60-e9846bbc52d0>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor[i, :len(ids)] = torch.tensor(labels[i])


In [79]:
# Define the model architecture using pytorch-lightning

import transformers
from transformers import CamembertForTokenClassification, CamembertConfig, AutoModelForTokenClassification, CamembertPreTrainedModel
from transformers.utils import ModelOutput

label2id = {
    "B-KEY": 1,
    "I-KEY": 2,
    "O": 0
  }
class CamembertForTokenClassification2(CamembertPreTrainedModel):


    def __init__(self, config):
        super().__init__(config)
        config = CamembertConfig.from_pretrained("yanekyuk/camembert-keyword-extractor")
        self.config = config
        self.config.num_labels = len(label2id)

        self.model = AutoModelForTokenClassification.from_pretrained("yanekyuk/camembert-keyword-extractor", config=self.config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = torch.nn.Dropout(classifier_dropout)
        self.classifier = torch.nn.Linear(config.hidden_size,config.num_labels)

        # Initialize weights and apply final processing
        self.post_init()

    def forward(self,input_ids = None,attention_mask = None, labels= None):
        """
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the token classification loss. Indices should be in `[0, ..., config.num_labels - 1]`.
        """


        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
        )

        sequence_output = outputs[0]

        #logits = self.classifier(sequence_output)
        logits = outputs.logits

        loss = None
        if labels is not None:
            # move labels to correct device to enable model parallelism
            labels = labels.to(logits.device)
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 3), labels.view(-1))


        return transformers.modeling_outputs.TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
# Define the model
device = torch.device('cuda')

config = CamembertConfig.from_pretrained("yanekyuk/camembert-keyword-extractor")
model = CamembertForTokenClassification2(config=config)
model.to(device)



from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

epochs = 20

def flat_accuracy(logits, labels):
    # Flatten the predictions and true values to 1 dimension
    flat_logits = torch.argmax(logits, dim=2).flatten()
    flat_labels = labels.flatten()

    # Calculate the accuracy
    return (flat_logits == flat_labels).float().mean()


for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)

    train_loss, train_accuracy = 0, 0
    nb_train_steps, nb_train_examples = 0, 0

    for step, batch in enumerate(train_dataloader):
        if step % 50 == 0 and not step == 0:
            print(f'Batch {step} of {len(train_dataloader)}')

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        optimizer.zero_grad()


        outputs = model(input_ids=b_input_ids, 
                        attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss
        logits = outputs.logits
        if loss is not None:
          train_loss += loss.item()
          loss.backward()

        optimizer.step()

        nb_train_examples += b_input_ids.size(0)
        nb_train_steps += 1

        train_accuracy += flat_accuracy(logits, b_labels)

    print(f'Training loss: {train_loss / nb_train_steps}')
    print(f'Training accuracy: {train_accuracy / nb_train_steps}')

    test_loss, test_accuracy = 0, 0
    nb_test_steps, nb_test_examples = 0, 0

    for batch in val_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(input_ids=b_input_ids, 
                        attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss
        logits = outputs.logits
        if loss is not None:
          test_loss += loss.item()

        test_accuracy += flat_accuracy(logits, b_labels)


        nb_test_examples += b_input_ids.size(0)
        nb_test_steps += 1

    print(f'Validation loss: {test_loss / nb_test_steps}')
    print(f'Validation accuracy: {test_accuracy / nb_test_steps}')



Epoch 1/20
----------


<ipython-input-74-f54ee28d1cc3>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_tensor[i, :len(ids)] = torch.tensor(ids)
<ipython-input-74-f54ee28d1cc3>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor[i, :len(ids)] = torch.tensor(labels[i])


Training loss: 0.10460487699234172
Training accuracy: 0.9780465960502625
Validation loss: 0.06466724053025245
Validation accuracy: 0.9872932434082031
Epoch 2/20
----------
Training loss: 0.0581430382781515
Training accuracy: 0.9832212328910828
Validation loss: 0.05101604908704758
Validation accuracy: 0.9869446754455566
Epoch 3/20
----------
Training loss: 0.036226922229520585
Training accuracy: 0.9870401620864868
Validation loss: 0.059096026420593264
Validation accuracy: 0.9816984534263611
Epoch 4/20
----------
Training loss: 0.019514814631915407
Training accuracy: 0.9943149089813232
Validation loss: 0.07239570170640945
Validation accuracy: 0.9869599342346191
Epoch 5/20
----------
Training loss: 0.010728703894480867
Training accuracy: 0.9974284172058105
Validation loss: 0.07245907820761203
Validation accuracy: 0.9859991073608398
Epoch 6/20
----------
Training loss: 0.007080836448279258
Training accuracy: 0.9985268115997314
Validation loss: 0.07585513181984424
Validation accuracy: 0.984

In [81]:
# Predict on a sample text

text = datax[1]['text']
# Tokenize the text

def extract_keywords(text,tokenizer, model):
  tokenized_text = tokenizer(text, padding=True, truncation=True, max_length=128*2, return_tensors='pt')

  # Make a prediction

  model.eval()

  with torch.no_grad():
      outputs = model(tokenized_text['input_ids'].to(device), tokenized_text['attention_mask'].to(device))

      # Get the predicted labels

      preds = torch.argmax(outputs.logits, dim=2).flatten()
      # Get the predicted keywords

      keywords = [tokenizer.decode(tokenized_text['input_ids'][0][i]) for i, pred in enumerate(preds) if pred == 1]

      return keywords

text = datax[1]['text']

print(extract_keywords(text,tokenizer, model))

['compte']


In [82]:
# Save the fine-tuned model

model_name = 'camembert-keywords-finetuned'
model.save_pretrained(model_name)
tokenizer.save_pretrained(model_name)


('camembert-keywords-finetuned/tokenizer_config.json',
 'camembert-keywords-finetuned/special_tokens_map.json',
 'camembert-keywords-finetuned/tokenizer.json')

In [86]:
# Load the fine-tuned model


model2 = CamembertForTokenClassification2.from_pretrained(model_name)
model2.to("cuda")
tokenizer2 = AutoTokenizer.from_pretrained(model_name)

# Extract 

print(extract_keywords(text,tokenizer2, model2))

['compte']


In [113]:
# Quantize the model

model_int8 = torch.ao.quantization.quantize_dynamic(
    model2,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8)  # the target dtype for quantized weights


# Save the quantized model
model_int8_tok = 'camembert-keywords-finetuned-quantized'
model_int8_name = model_int8_tok+'/pytorch_model.bin'
torch.save(model_int8.state_dict(), model_int8_name)
tokenizer2.save_pretrained(model_int8_tok)


('camembert-keywords-finetuned-quantized/tokenizer_config.json',
 'camembert-keywords-finetuned-quantized/special_tokens_map.json',
 'camembert-keywords-finetuned-quantized/tokenizer.json')

In [114]:
# Load the quantized model
model_int8_tok = 'camembert-keywords-finetuned-quantized'
model_name = 'camembert-keywords-finetuned'

loaded_model = CamembertForTokenClassification2.from_pretrained(model_int8_tok)
loaded_state_dict = torch.load(model_int8_name)
loaded_model.load_state_dict(loaded_state_dict,strict=False)
loaded_model.to('cuda')

tokenizer3 = AutoTokenizer.from_pretrained(model_int8_tok)

# Extract keywords

print(extract_keywords(text,tokenizer3, loaded_model))

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:314: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,
Some weights of the model checkpoint at camembert-keywords-finetuned-quantized were not used when initializing CamembertForTokenClassification2: ['model.roberta.encoder.layer.6.intermediate.dense._packed_params.dtype', 'model.roberta.encoder.layer.5.attention.self.key.zero_point', 'model.roberta.encoder.layer.11.output.dense._packed_params.dtype', 'model.roberta.encoder.layer.0.attention.self.key._packed_params._packed_params', 'model.roberta.encoder.layer.2.attention.self.query._packed_params._packed_params', 'model.roberta.encoder.layer.6.attention.output.dense._packed_params.dtype', 'model.roberta.encoder.layer.3.inter

['entreprise']


In [115]:
def evaluate(modelx):
  test_loss, test_accuracy = 0, 0
  nb_test_steps, nb_test_examples = 0, 0

  for batch in val_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch

      with torch.no_grad():
          outputs = modelx(input_ids=b_input_ids, 
                      attention_mask=b_input_mask, labels=b_labels)

      loss = outputs.loss
      logits = outputs.logits
      if loss is not None:
        test_loss += loss.item()

      test_accuracy += flat_accuracy(logits, b_labels)


      nb_test_examples += b_input_ids.size(0)
      nb_test_steps += 1


  print(f'Validation loss: {test_loss / nb_test_steps}')
  print(f'Validation accuracy: {test_accuracy / nb_test_steps}')

In [116]:
evaluate(loaded_model)

<ipython-input-74-f54ee28d1cc3>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_tensor[i, :len(ids)] = torch.tensor(ids)
<ipython-input-74-f54ee28d1cc3>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor[i, :len(ids)] = torch.tensor(labels[i])


Validation loss: 0.2880912125110626
Validation accuracy: 0.9240261912345886


In [118]:
import shutil
shutil.make_archive("camembert-keywords-finetuned-quantized", 'zip', "camembert-keywords-finetuned-quantized")
shutil.make_archive("camembert-keywords-finetuned", 'zip', "camembert-keywords-finetuned")

'/content/camembert-keywords-finetuned.zip'

In [119]:
from google.colab import files
files.download("camembert-keywords-finetuned-quantized.zip")
files.download("camembert-keywords-finetuned.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>